## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Input parameters

In [17]:
trailname = 'gr131' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta = 0.005 # Tolerance around bounding box per trail section [deg]

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Loading GPX file

In [3]:
trail = gr_utils.get_gpx(trailname)

Loading trail points from <data_input/gr131.gpx>...
Finished loading.


## Gathering ROAD data from OSM network

In [4]:
filename_roads = 'cache/' + trailname + '_roads.csv'

if not os.path.isfile(filename_roads): # The merged ROADS file does not exist, construct it
    
    print('Merged ROADS file was not found, merging and saving...')
    gr_mapmatch.trail2roads(trailname, trail, points_per_batch, delta) # Main batch processor
    data_roads_raw = gr_utils.merge_roads(trailname, trail, points_per_batch) # Merge the different sections
    data_roads = gr_mapmatch.remove_repeat_segments(data_roads_raw) # Remove backtracked sections
    gr_utils.write_roads(trailname, data_roads) # Write the merged ROADS data
#     print('Saved.')

else: # The merged ROADS file does exist, just load it
    
    print('Loading merged ROADS file...')
    data_roads = gr_utils.read_roads(trailname) # Read the merged sections
    print('Loaded.')

Loading merged ROADS file...
Loaded.


## Gathering PLACE data from OSM network

In [5]:
filename_places = 'cache/' + trailname + '_places.csv'

if not os.path.isfile(filename_places): # The merged PLACES file does not exist
    
    print('Merged PLACES file was not found, merging and saving...')
    gr_placematch.roads2places(trailname,data_roads,points_per_batch_places, delta_places, buffersize, tol_area)
    data_places = gr_utils.merge_places(trailname, data_roads, points_per_batch_places) # Merge the different sections
    gr_utils.write_places(trailname, data_places) # Write the merged PLACES data
    print('Saved.')
    
else: # The merged PLACES file does exist
    
    print('Loading merged PLACES file...')
    data_places = gr_utils.read_places(trailname) # Read the merged PLACES file
    print('Loaded.')

Merged PLACES file was not found, merging and saving...
Handling batch 0 of 41 that covers road segments 0 through 99...
There are 77 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 1 of 41 that covers road segments 100 through 199...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 113 uncorrected landuse places
There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 2 of 41 that covers road segments 200 through 299...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 370 uncorrected landuse places


...

   Writing outputs to file...
   Finished this batch.
Handling batch 40 of 41 that covers road segments 4000 through 4099...
There are 473 uncorrected landuse places


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 41 of 41 that covers road segments 4100 through 4168...
There are 173 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


Saved.


## Evaluating PROCESSED properties

In [18]:
filename_processed = 'data_output/' + trailname + '_processed.csv'

if not os.path.isfile(filename_processed): # The PROCESSED file does not exist
    
    print('The PROCESSED file was not found, processing and saving...')
    data = gr_process.places2processed(data_places,tol_d,types_slow,types_heavy,tracktype_p0,tracktype_p1,tracktype_p2,surface_p0,surface_p1,highway_p1) # Determine traffic/development/paved status & GR types
    gr_utils.write_processed(trailname, data) # Write the processed data
    print('Saved.')
    
else: # The PROCESSED file does exist
    
    print('The PROCESSED file already exists, so nothing needs to be done.')

The PROCESSED file was not found, processing and saving...
Saved.
